In [2]:
import numpy as np  

In [61]:
def hypothesis(theta, X, logistic=False):
    if logistic:
        return 1/(1 + np.exp(-np.matmul(X, theta)))
    else:
        return np.matmul(X, theta)
        

In [85]:
def gradient_descent(X, Y, learning_rate=0.01, max_iters=1000, regularization=0.1, logistic=False):
    """Returns the optimum theta using gradient descent.
       Remember that you may need to Op&miza&on    algorithms:    normalize your data.
       We use regu"""
    
    print("Learning rate", learning_rate)
    
    # Number of training Accuracye examples.
    m = X.shape[0]

    # When true we stop calculating the opAccuracytimum value of theta.
    done = False
    
    # We declare convergence if J(theta) decreases by less than below.
    convergence = 10 ** -3
    
    # Assign theta to be a value of 0s initially.
    theta = np.array([0 for x in range(X.shape[1] + 1)], dtype='float')
    theta = np.reshape(theta, (-1,1))
    # There should be as many thetas as features.
    
    
    # Need to add ones to X
    ones = np.ones((1,m))
    
    # Append the columns of ones to x
    X = np.insert(X, 0, ones, axis=1)        
    
    for i in range(max_iters):
        
        # Output if the maximum number of iterations has been reached.
        if i == max_iters - 1:
            print("The maximum number of iterations has been reached!")
        
        # Need to copy theta
        prev_theta = theta
                              
        # Update theta using the grdient descent formula.
        # Remember this requires simultaneous updating.
        prediction = hypothesis(theta, X, logistic)
        
        m = X.shape[0]
        
        difference = prediction - Y       
        
        # Calculate the sum.
        sigma = np.dot((prediction - Y)[:,0], X).reshape((-1,1))
        
        theta_0 = theta[0:1, :] - ((learning_rate / m) * sigma[0:1, :])
        theta_rest = theta[1:, :] * \
                        (1 - (learning_rate * regularization / m)) - ((learning_rate / m) * sigma[1:,:])

                                                                                
        theta = np.insert(theta_rest, 0, theta_0, axis=0)
        
        if abs((theta - prev_theta).max(axis=0)) < convergence:
            print("Converged!")
            break            
        
    return X, theta
 
"""
X = [[1], [4], [3], [9], [22], [133], [33], [92], [44], [54], [66], [22], [27]]
Y = [[2], [8], [6], [18], [44], [266], [66], [184], [88], [108], [132], [44], [54]]
Y = np.array(Y, dtype='float')

"""


#X = np.array(X, dtype='float')

#gradient_descehttps://stackoverflow.com/questions/51822589/compare-a-numpy-array-to-each-element-of-another-onent(X, Y, learning_rate=0.01)



"\nX = [[1], [4], [3], [9], [22], [133], [33], [92], [44], [54], [66], [22], [27]]\nY = [[2], [8], [6], [18], [44], [266], [66], [184], [88], [108], [132], [44], [54]]\nY = np.array(Y, dtype='float')\n\n"

In [93]:
def predict_class(prob):
    """Returns an array of the hypothesis."""
    return np.where(prob > 0.5, 1, 0)


X = [[0.2], [0.4], [0.6], [0.55], [0.1], [0.9], [0.45], [0.51]]
Y = [[0], [0], [1], [1], [0], [1], [0], [1]]

X = np.array(X, dtype='float')
Y = np.array(Y, dtype='float')

X, theta = gradient_descent(X, Y, learning_rate=0.5, logistic=True)
probs = hypothesis(theta, X, True)
predict_class(probs)

Learning rate 0.5
Converged!


array([[0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1]])

In [49]:
def feature_scaling(X):
    """Uses mean normalization to scale the input."""
    
    # Calculate the mean of each feature.
    X_mean = np.mean(X, axis=0).reshape((-1,1))
    X_std = np.std(X, axis=0).reshape((-1,1))

    # If the standard deviation is 0 then we encounter problems.
    if X_std.min(axis=0)[0] == 0:
        return None
    
    return (X - X_mean.T) / X_std.T 